In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import re
pd.set_option('display.max_columns', 999)

In [4]:
db = sqlite3.connect('data/NPPES_Data_Dissemination_February_2021/npidata2.sqlite')
pd.read_csv('data/nashville_referrals_normalised.csv').to_sql('filtered_hop', db, if_exists = 'append', index = False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [118]:
query = """
WITH CTE AS (SELECT 
	from_npi
FROM filtered_hop 
WHERE  to_facility_group LIKE '%Vanderbilt%' 
)
SELECT 
	from_npi,
	Provider_Last_Name,
	Provider_First_Name,
	COUNT(DISTINCT to_facility_group) AS Group_Count,
	SUM(patient_count) AS Total_referrals,
    GROUP_CONCAT(DISTINCT to_facility_group) AS Groups
FROM filtered_hop 
JOIN npidata ON NPI = from_npi
JOIN CBSA ON Zip = Address_Postal_Codes
WHERE from_npi NOT IN (
	SELECT from_npi
	FROM CTE)
GROUP BY from_npi, Provider_Last_Name, Provider_First_Name
HAVING COUNT(DISTINCT to_facility_group) > 1
"""

In [119]:
no_vandy = pd.read_sql(query, db) 

In [120]:
no_vandy

,from_npi,Provider_Last_Name,Provider_First_Name,Group_Count,Total_referrals,Groups
0,1003819277,CAGE,JOHN,4,9894,"HCA,Williamson Medical Center,Ascension Saint ..."
1,1013233865,COLEMAN,CATHERINE,2,4884,"HCA,Ascension Saint Thomas"
2,1023004041,FREUND,LAWRENCE,2,9792,"Maury Regional Medical Center,Ascension Saint ..."
3,1033103551,STEIER,JILL,2,4194,"HCA,Sumner Regional Medical Center"
4,1033313127,GERDES,JODI,2,5496,"HCA,Sumner Regional Medical Center"
...,...,...,...,...,...,...
206,1972658060,KETCH,TERRY,2,73584,"HCA,NorthCrest Medical Center"
207,1982836672,DARKS,TINA,2,8838,"HCA,Sumner Regional Medical Center"
208,1992150940,SCALF,MARY,2,2970,"HCA,Sumner Regional Medical Center"
209,1992703698,EDWARDS,WILLIAM,2,7050,"Ascension Saint Thomas,Sumner Regional Medical..."


In [71]:
no_vandy = no_vandy.drop_duplicates()


In [72]:
no_vandy

,Phys_NPI,Phys_Class,Phys_Last_Name,Phys_First_Name,Hosp_NPI,Hospital,pt_count
0,1437154002,Anesthesiology,BURRICHTER,CALVIN,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",209
2,1003813874,Internal Medicine,WEBB,KEITH,1518346469,"SAINT THOMAS RIVER PARK HOSPITAL, LLC",176
4,1336147776,Pediatrics,WEBB,SONJA,1518346469,"SAINT THOMAS RIVER PARK HOSPITAL, LLC",106
6,1114927936,"Nurse Anesthetist, Certified Registered",RISCHBECK,TONYA,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",106
8,1083615934,Thoracic Surgery (Cardiothoracic Vascular Surg...,WHEATLEY,GRAYSON,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",190
...,...,...,...,...,...,...,...
156,1225494891,"Nurse Anesthetist, Certified Registered",EMBERTON,NICOLE,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",148
158,1588117824,"Nurse Anesthetist, Certified Registered",SLOOP,SCOTT,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",134
160,1548715147,Nurse Practitioner,LANE,CATHERINE,1629025648,SAINT THOMAS WEST HOSPITAL,132
162,1841748423,"Nurse Anesthetist, Certified Registered",HOUSE,NATALIE,1629025648,SAINT THOMAS WEST HOSPITAL,106


In [73]:
no_vandy['Hospital'].unique()

array(['HCA HEALTH SERVICES OF TENNESSEE, INC.',
       'SAINT THOMAS RIVER PARK HOSPITAL, LLC', 'MAURY REGIONAL HOSPITAL',
       'SUMNER REGIONAL MEDICAL CENTER LLC',
       'HENDERSONVILLE HOSPITAL CORPORATION',
       'HTI MEMORIAL HOSPITAL CORPORATION', 'SAINT THOMAS WEST HOSPITAL',
       'SAINT THOMAS RUTHERFORD HOSPITAL',
       'CENTRAL TENNESSEE HOSPITAL CORPORATION',
       'SHELBYVILLE HOSPITAL COMPANY LLC', 'NORTHCREST MEDICAL CENTER'],
      dtype=object)

In [74]:
Hospitals = ["HCA HEALTH SERVICES OF TENNESSEE, INC.","SAINT THOMAS WEST HOSPITAL","CENTRAL TENNESSEE HOSPITAL CORPORATION"]
no_vandy = no_vandy[no_vandy["Hospital"].isin(Hospitals)]

In [77]:
no_vandy.sort_values(by=["pt_count"],ascending=False)

,Phys_NPI,Phys_Class,Phys_Last_Name,Phys_First_Name,Hosp_NPI,Hospital,pt_count
164,1124578687,Nurse Practitioner,LOWE,LINDSAY,1629025648,SAINT THOMAS WEST HOSPITAL,222
0,1437154002,Anesthesiology,BURRICHTER,CALVIN,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",209
150,1972904035,Nurse Practitioner,PENICK,DANIEL,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",200
8,1083615934,Thoracic Surgery (Cardiothoracic Vascular Surg...,WHEATLEY,GRAYSON,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",190
120,1831435817,Nurse Practitioner,CHRISTIAN,KIMBERLY,1780778969,SAINT THOMAS WEST HOSPITAL,186
44,1710914148,Nurse Practitioner,MITCHELL,SUSAN,1629025648,SAINT THOMAS WEST HOSPITAL,183
152,1841694346,"Nurse Anesthetist, Certified Registered",KENSINGER,MARIBETH,1629025648,SAINT THOMAS WEST HOSPITAL,167
154,1376928986,Nurse Practitioner,RELLINGER,BRANDIANN,1629025648,SAINT THOMAS WEST HOSPITAL,161
60,1114001229,Internal Medicine,MACMURDO-FRANCE,CHRISTINA,1629025648,SAINT THOMAS WEST HOSPITAL,160
112,1508139593,Nurse Practitioner,PFEFFER,KENDRA,1629025648,SAINT THOMAS WEST HOSPITAL,158


In [ ]:
no_vandy['Phys_Class'].unique()